In [69]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import plotly.express as px
from sklearn.cluster import KMeans

In [62]:
adp = pd.read_csv("data/fantasypros/adp/HALF_PPR_ADP.csv")
adp.dropna(subset=["PLAYER"], inplace=True)
adp

,Unnamed: 0,PLAYER,POS,AVG
0,0,Christian McCaffrey,RB,1.0
1,1,Saquon Barkley,RB,2.0
2,2,Ezekiel Elliott,RB,3.0
3,3,Michael Thomas,WR,5.0
4,4,Alvin Kamara,RB,5.0
...,...,...,...,...
376,376,Adam Trautman,TE,380.0
377,377,Trent Taylor,WR,381.0
378,378,Corey Clement,RB,382.0
379,379,Devine Ozigbo,RB,383.0


In [64]:
data = pd.read_csv("data/yearly/2019.csv")
data.dropna(subset=["Pos"], inplace=True)

In [117]:
def plotKmeans(df, x="ADP", y="FantasyPoints", position="RB", clusters=2):
    dataset = df.dropna(subset=[x, y])
    dataset = dataset[dataset["Pos"] == position]
    points = list(zip(dataset[x].to_list(), dataset[y].to_list()))
    kmeans = KMeans(n_clusters=clusters, random_state=42) 
    kmeans.fit(points)
    kmeans.labels_
    dataset["labels"] = kmeans.labels_
    fig = px.scatter(dataset, x=x, y=y, color="labels", text="Player")
    print(x)
    fig.update_layout({
        "font" : dict(
        family="sans serif",
        size=18,
        color="LightSeaGreen"
    ),
        "xaxis_title": x,
        "yaxis_title": y,
    })
    fig.show()

In [65]:
xs = []
ys = []
pos = []
data["ADP"] = pd.Series()
for i, player in adp.iterrows():  
    try:
        ix = process.extractOne(player["PLAYER"], data["Player"])
        data["ADP"].loc[ix[2]] = player["AVG"]
    except:
        print(player["PLAYER"])
    

/var/folders/zd/hd98t63j70j07gdtg557z0980000gp/T/ipykernel_86027/1954686457.py:4: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/var/folders/zd/hd98t63j70j07gdtg557z0980000gp/T/ipykernel_86027/1954686457.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [66]:
px.scatter(data, x="ADP", y="FantasyPoints", color="Pos", text="Player").show()

In [118]:
plotKmeans(data, position="WR", clusters=5)

ADP
